In [1]:
import sys
sys.path.append('./yolo')
from detection_fusion import EnsembleModel
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
from rcnn.dataset import ChestCocoDetection
from torchvision import transforms
import matplotlib.patches as patches
from torch.utils.data import DataLoader
import tqdm

In [2]:
def add_bounding_boxes(target, ax):
    boxes = target['boxes']
    for box in boxes:
        mp_box = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], edgecolor="r", facecolor='none')
        ax.add_patch(mp_box)

def show_samples_for(test, train):
    fig, axs = plt.subplots(1, 2, figsize=(10,8))
    #fig.suptitle(f'Study: {samples["id"].iloc[i]}')
    for im_i, (img, data) in enumerate([test, train]):
        ax = axs[im_i] if isinstance(axs, np.ndarray) else axs
        ax.set_title(f'Instance / Image Nr. {im_i + 1} / {2}')
        #dcm = pydicom.dcmread(image_path)
        plt.figure()
        ax.imshow(img.permute(1,2,0))#,cmap=plt.cm.bone)
        add_bounding_boxes(data, ax)
    fig.savefig("combo.jpg")

In [ ]:
test = ChestCocoDetection(root="F:\\aml-project\data\\siim-covid19-detection", ann_file="F:\\aml-project\\data\\siim-covid19-detection\\test.json", training=False)

In [ ]:
idx = np.random.randint(len(test))
test_img = test.__getitem__(idx)
print(type(test_img[0]), test_img[0].shape)
print(test_img[1])
img = test_img[0]
img_pil = transforms.ToPILImage()(img)

In [ ]:
results, boxes, scores = detection_fusion(img_pil)

In [ ]:
show_samples_for(test_img, (test_img[0], {"boxes":boxes}))

### Evaluation of Ensemble method

In [3]:
from yolo.utils.general import xywh2xyxy, scale_coords, box_iou
from yolo.utils.metrics import ap_per_class
import torch
from rcnn.model import ChestRCNN
from yolo.yolo import Model

In [4]:
def collate_fn(batch):
    imgs, targets = zip(*batch)
    imgs = torch.stack(imgs)
    return tuple((imgs, targets))

In [5]:
# need bs = 1 because model fusion cannot handle batches 
test = ChestCocoDetection(root="F:\\aml-project\data\\siim-covid19-detection", ann_file="F:\\aml-project\\data\\siim-covid19-detection\\test.json", training=False)
test_loader = DataLoader(test, batch_size=1, shuffle=False, pin_memory=False, num_workers=0, collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [6]:
YOLO_FINAL_MODEL_PATH = "./yolo/models_final_giou_40/yolov5_epoch_25.pt"
FASTER_RCNN_FINAL_MODEL_PATH = "./rcnn/models/fasterrcnn_epoch_23.pt"
RESNET_BACKBONE_PATH = "./resnet/models/resnext101_32x8d_epoch_35.pt"

yolov5_weights = torch.load(YOLO_FINAL_MODEL_PATH)
fasterrcnn_r101_weights = torch.load(FASTER_RCNN_FINAL_MODEL_PATH)

yolo = Model(cfg="./yolo/yolo5l.yaml",ch=3,nc=1)
yolo.load_state_dict(yolov5_weights, strict=False) 

fasterRcnn = ChestRCNN(RESNET_BACKBONE_PATH)
fasterRcnn.load_state_dict(fasterrcnn_r101_weights)
ensemble = EnsembleModel(fasterRcnn=fasterRcnn, yolo=yolo)

D:\Anaconda\envs\aml\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [9]:
jdict, stats, ap, ap_class = [], [], [], []
stats_yolo, stats_fcnn = [], []
iouv = torch.linspace(0.5, 0.55, 1) # iou vector for mAP@0.5:0.95
niou = iouv.numel()
general_test_results = []
for i, (images, targets) in enumerate(tqdm.tqdm(test_loader)):
    nb, _, height, width = images.shape
    assert nb == 1
    img_pil = transforms.ToPILImage()(images[0])
    _, boxes, scores, pred_labels, yolo_box, yolo_scores, yolo_labels, fcnn_box, fcnn_scores, fcnn_labels = ensemble.detection_fusion(img_pil, extended_output=True)
    boxes = torch.tensor(boxes)
    yolo_box = torch.tensor(yolo_box)
    fcnn_box = torch.tensor(fcnn_box)
    gt_boxes = targets[0]['boxes']
    labels = targets[0]['labels']
    nl = len(labels)
    tcls = labels[:].tolist() if nl else []  # target class

    if len(boxes) == 0:
        if nl:
            stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
            stats_yolo.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
            stats_fcnn.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))

        continue

    correct = torch.zeros(len(boxes), niou, dtype=torch.bool)
    correct_yolo = torch.zeros(len(yolo_box), niou, dtype=torch.bool)
    correct_fcnn = torch.zeros(len(fcnn_box), niou, dtype=torch.bool)
    
    if nl:
        detected = []
        detected_yolo = []
        detected_fcnn = []
        tcls_tensor = labels[:]

        # Per target class
        for cls in torch.tensor([1]):
            ti = (cls == tcls_tensor).nonzero(as_tuple=False).view(-1)
            pi = (cls == torch.tensor(pred_labels)).nonzero(as_tuple=False).view(-1)
            pi_yolo = (cls == torch.tensor(yolo_labels)).nonzero(as_tuple=False).view(-1)
            pi_fcnn = (cls == torch.tensor(fcnn_labels)).nonzero(as_tuple=False).view(-1)

            # Search for detections
            if pi.shape[0]:
                ious, i = box_iou(boxes[pi], gt_boxes[ti]).max(1)  # best ious, indices
                # Append detections - Ensemble
                detected_set = set()
                for j in (ious > iouv[0]).nonzero(as_tuple=False):
                    d = ti[i[j]]  
                    if d.item() not in detected_set:
                        detected_set.add(d.item())
                        detected.append(d)
                        correct[pi[j]] = ious[j] > iouv  
                        if len(detected) == nl: 
                            break
                            
            if pi_yolo.shape[0]:
                ious_yolo, i_yolo = box_iou(yolo_box[pi_yolo], gt_boxes[ti]).max(1)
                # Yolo
                detected_set_yolo = set()
                for j in (ious_yolo > iouv[0]).nonzero(as_tuple=False):
                    d = ti[i_yolo[j]]  
                    if d.item() not in detected_set_yolo:
                        detected_set_yolo.add(d.item())
                        detected_yolo.append(d)
                        correct_yolo[pi_yolo[j]] = ious[j] > iouv  
                        if len(detected_yolo) == nl: 
                            break
                            
            if pi_fcnn.shape[0]:
                ious_fcnn, i_fcnn = box_iou(fcnn_box[pi_fcnn], gt_boxes[ti]).max(1)
                # Faster R-CNN
                detected_set_fcnn = set()
                for j in (ious_fcnn > iouv[0]).nonzero(as_tuple=False):
                    d = ti[i_fcnn[j]]  
                    if d.item() not in detected_set_fcnn:
                        detected_set_fcnn.add(d.item())
                        detected_fcnn.append(d)
                        correct_fcnn[pi_fcnn[j]] = ious[j] > iouv  
                        if len(detected_fcnn) == nl: 
                            break



        # Append statistics (correct, conf, pcls, tcls)
        #print((correct.cpu(), torch.tensor(scores),torch.tensor(pred_labels), tcls))
        stats.append((correct.cpu(), torch.tensor(scores),torch.tensor(pred_labels), tcls))
        stats_yolo.append((correct_yolo.cpu(), torch.tensor(yolo_scores),torch.tensor(yolo_labels), tcls))
        stats_fcnn.append((correct_fcnn.cpu(), torch.tensor(fcnn_scores),torch.tensor(fcnn_labels), tcls))

for statistic in [stats, stats_yolo, stats_fcnn]:
    statistic = [np.concatenate(x, 0) for x in zip(*statistic)] 
    if len(statistic) and statistic[0].any():
        p, r, ap, f1, ap_class = ap_per_class(*statistic)
        ap50, ap = ap[:, 0], ap.mean(1) 
        mp, mr, map50, map = p.mean(), r.mean(), ap50.mean(), ap.mean()
        nt = np.bincount(statistic[3].astype(np.int64), minlength=1)

        general_test_results.append({
        "precision": p,
        "recall": r,
        "ap": ap,
        "f1": f1,
        "ap_class": ap_class,
        "ap": ap,
        "ap50": ap50,
        "mp": mp,
        "mr": mr,
        "map50": map50,
        "map": map
    })

    else:
        nt = torch.zeros(1)


  0%|                                                                                  | 1/859 [00:04<57:50,  4.05s/it]

tensor([[3.84430e+02, 1.83289e+02, 4.81437e+02, 3.53334e+02, 3.59826e-01, 0.00000e+00],
        [1.22471e+02, 1.54861e+02, 2.25946e+02, 3.46088e+02, 2.74667e-01, 0.00000e+00]])


  0%|▏                                                                                 | 2/859 [00:08<57:15,  4.01s/it]

tensor([[8.47206e+01, 1.06839e+02, 2.28598e+02, 4.02877e+02, 3.09484e-01, 0.00000e+00],
        [3.08120e+02, 9.64006e+01, 4.42179e+02, 3.92486e+02, 1.62329e-01, 0.00000e+00]])


  0%|▎                                                                                 | 3/859 [00:11<56:50,  3.98s/it]

...

tensor([[4.20792e+01, 1.00288e+02, 2.02641e+02, 4.21484e+02, 3.79805e-01, 0.00000e+00],
        [2.88122e+02, 8.53361e+01, 4.60687e+02, 4.26115e+02, 3.21656e-01, 0.00000e+00]])


100%|███████████████████████████████████████████████████████████████████████████████▉| 858/859 [57:53<00:04,  4.02s/it]

tensor([[1.03928e+02, 8.67695e+01, 2.37082e+02, 3.50228e+02, 2.95001e-01, 0.00000e+00],
        [3.08233e+02, 7.87307e+01, 4.54369e+02, 3.97903e+02, 2.90639e-01, 0.00000e+00]])


100%|████████████████████████████████████████████████████████████████████████████████| 859/859 [57:57<00:00,  4.05s/it]

tensor([[3.19278e+02, 2.75469e+02, 4.43503e+02, 4.60951e+02, 1.20839e-01, 0.00000e+00]])


In [11]:
general_test_results[0]

{'precision': array([    0.32703]),
 'recall': array([    0.40346]),
 'ap': array([    0.22222]),
 'f1': array([    0.36124]),
 'ap_class': array([1]),
 'ap50': array([    0.22222]),
 'mp': 0.32702590011204447,
 'mr': 0.4034580034580035,
 'map50': 0.22222274442615636,
 'map': 0.22222274442615636}

In [12]:
general_test_results[1]

{'precision': array([    0.28628]),
 'recall': array([    0.29299]),
 'ap': array([    0.14523]),
 'f1': array([     0.2896]),
 'ap_class': array([1]),
 'ap50': array([    0.14523]),
 'mp': 0.28627978695647754,
 'mr': 0.2929908510027988,
 'map50': 0.1452307559503463,
 'map': 0.1452307559503463}

In [13]:
general_test_results[2]

{'precision': array([    0.27606]),
 'recall': array([    0.27209]),
 'ap': array([   0.096146]),
 'f1': array([    0.27406]),
 'ap_class': array([1]),
 'ap50': array([   0.096146]),
 'mp': 0.27605637751810674,
 'mr': 0.27209154481881753,
 'map50': 0.0961461165073274,
 'map': 0.0961461165073274}